<div style="margin: 0 auto 30px; height: 60px; border: 2px solid gray; border-radius: 6px;">
  <div style="float: left;"><img src="assets/epfl.png" /></div>
  <div style="float: right; margin: 20px 30px 0; font-size: 10pt; font-weight: bold;"><a href="https://moodle.epfl.ch/course/view.php?id=18253">COM202 - Signal Processing</a></div>
</div>
<div style="clear: both; font-size: 30pt; font-weight: bold; color: #483D8B;">
    Lab 2.A: understanding the discrete Fourier transform
</div>

In this two-part notebook we will study the properties and the applications of the Discrete Fourier Transform (DFT), arguably the most important analysis tool in digital signal processing. 

In this first part we will incrementally work our way to the definition of the DFT by showing
 * we will introduce the idea of signal decomposition, where we express a signal as the sum of elementary building blocks
 * we will show by example how to build signals using harmonic sines and cosines as building blocks
 * we will move to the more efficient family of complex-valued harmonic oscillation (the Fourier basis)
 * we will explore the properties of the Fourier basis

At the end of this lab you should be able to 
- understand that any signal can be built as a linear combination of sinusoids
- understand the symmetries and periodicities of the Fourier basic elements

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as wg

plt.rcParams["figure.figsize"] = (12,3)

In [ ]:
# These are helper functions for the interactive plots used later in the notebook. No need to study or understand
#  the code unless you are interested in advanced plotting and interactive widgets

UC_LIM = 1.4
UC_ARW = 0.1
UC_ARC = 0.8
UC_TXT = 1.2
UC_PSZ = 5

def wcolor(w):
    if np.isclose(w, 0) or np.isclose(w, np.pi):
        return 'C0'
    elif w > 0:
        return 'C2'
    else:
        return 'C3'
    
def arc(a=0, b=2*np.pi, r=1, pts=60, col='C0', arrow=False):
    theta = np.linspace(a, b, pts)
    plt.plot(r * np.cos(theta), r * np.sin(theta), color=col, linewidth=2)
    if arrow:
        d = np.sign(b) * UC_ARW
        x = r * np.array([np.cos(b-d), np.sin(b-d)])
        y = r * np.array([np.cos(b), np.sin(b)]) - x
        plt.arrow(*x, *y, shape='full', lw=1, length_includes_head=True, head_width=.08, color=col)

def ucsetup():
    plt.gcf().set_figheight(UC_PSZ)
    plt.gca().set_aspect('equal', 'box') 
    plt.gca().set(xlim=(-UC_LIM, UC_LIM), ylim=(-UC_LIM, UC_LIM))
    plt.axvline(0, linewidth=0.8, color='k')
    plt.axhline(0, linewidth=0.8, color='k')
    plt.axis('off')  
    arc()
    
def ucpoint(w, lb='', col='C2'):
    x, y = np.cos(w), np.sin(w)
    plt.plot([0, x], [0, y], color=col)
    plt.plot(x, y, 'o', color=col)
    plt.text(UC_TXT * x, UC_TXT * y, lb, ha='center', va='bottom' if np.round(y)==0 else 'center')          

def ctspin(w):
    w = np.fmod(w, 2 * np.pi)
    ucsetup()
    ucpoint(w, f'$e^{{j\\theta}}$', wcolor(w))
    arc(0, w, r=UC_ARC, col='C1', arrow=True)
    return w

def dtspin(N, k, n=None):
    ucsetup()
    w = 2 * np.pi / N * k
    for m in range(n+1 if n is not None else N):
        ucpoint(w * m, f'$w_{{{k}}}[{m}]$', wcolor(np.pi - w))
    ab = [0, w] if n is None else [w * (n-1), w * n]
    arc(*ab, r=UC_ARC, col='C1', arrow=True)

def ccspin(N, k, n=1):
    ucsetup()
    w = 2 * np.pi / N * k * n
    ucpoint(w, f'$w_{{{k}}}[{n}]$')
    ucpoint(-w, f'$w_{{{N-k}}}[{n}]$', 'C3')
    plt.plot([np.cos(w), np.cos(w)], [np.sin(w), -np.sin(w)], 'C6')
    plt.plot([np.cos(w), np.cos(w)], [np.sin(w), -np.sin(w)], 'oC6')
    plt.plot([0, np.cos(w)], [0, 0], 'C9')
    plt.plot(np.cos(w), 0, 'oC9')
  
def show_cex(N:int, k:int):
    w = 2 * np.pi / N * k
    cex = np.round(np.exp(1j * w * np.arange(N)), 3)
    plt.subplot(1,2,1)
    ucsetup()
    arc(0, w, r=UC_ARC, col='C1', arrow=True)    
    pts = {np.round(np.fmod(w * n, 2 * np.pi), 2): n for n in range(N-1,-1,-1)}
    for p, n in pts.items():
        ucpoint(n * w, f'$w_{{{k}}}[{n}]$', 'C2' if n < 2 else 'lightgray')
    plt.subplot(2,2,2)
    plt.stem(cex.real)
    plt.title(f'$\\mathrm{{Re}}\\{{\\mathbf{{w}}_{{{k}}}\\}} = \\mathbf{{c}}_{{{k}}}$')
    plt.subplot(2,2,4)
    plt.stem(cex.imag)
    plt.title(f'$\\mathrm{{Im}}\\{{\\mathbf{{w}}_{{{k}}}\\}} = \\mathbf{{s}}_{{{k}}}$')
    plt.tight_layout()

def show_freqs(N:int):
    ucsetup()    
    for k in range(N):
        w = 2 * np.pi / N * k
        if k < N / 2:
            ucpoint(w, f'$\\omega_{{{k}}}$', wcolor(w))
        elif k == N / 2:
            ucpoint(np.pi, f'$\\omega_{{{k}}} = \\pi$', wcolor(0))
        else:
            ucpoint(w, f'$\\omega_{{{k}}} = -\\omega_{{{N-k}}}$', wcolor(np.pi - w))
    plt.title(f'$N = {N}$')

def display_sbs(interactive_widget):
    c = interactive_widget.children
    controls = wg.HBox(c[:-1], layout=wg.Layout(align_items='center'))
    return display(wg.HBox([controls, c[-1]]))

# Introduction

The fundamental intuition of Fourier analysis is that a signal, no matter how complicated its "shape", can always be represented as a weighted sum of sinusoidal oscillations; a Fourier transform is the method to compute the value of the necessary weights. 

In this notebook, all signals are going to be discrete-time signals of finite length $N$; unless otherwise noted, **we will assume $N$ to be an even integer**. A length-$N$ signal $\mathbf{x}$ is simply an array of $N$ complex values; just like in C or Python, the individual samples will be denoted with an integer index within square brackets:
$$
  x[n], \qquad n = 0, 1, \ldots, N-1.
$$

## Warmup: two test signals

In the rest of this notebook we will use two simple test signals, defined below. These signals are uniquely defined for $0 \le n < N$ but they can also be considered as periodic signals with period $N$; this will be useful when we will express them as the sum of $N$-periodic sinusoids.

### A discrete-time square wave

A balanced, discrete-time square wave with period $N$ samples is defined over $[0, N-1]$ as
$$
    q[n] = \begin{cases} 0 & n =0, N/2 \\ 1 & 0 < n < N/2 \\ -1 & N/2 < n < N \end{cases}
$$
and prolonged by periodicity over $\mathbb{Z}$. We assume $N$ even, so that $N/2$ is an integer; the square wave is antisymmetric around its midpoint $N/2$, hence the adjective of "balanced".

In Python, we can easily implement a generator for this signal using the modulo operation:

In [ ]:
def sqw(N: int, n: np.ndarray=None) -> np.ndarray:
    # if no index range then return the values over [0, N-1]    
    n = np.arange(N) if n is None else n
    q = np.ones_like(n).astype(float)
    q[np.mod(n, N/2) == 0] = 0
    q[np.mod(n, N) > N/2] *= -1
    return q

Here is the first period of the square wave

In [ ]:
N = 64
plt.stem(sqw(N));

but we can also plot as many periods as we like:

In [ ]:
P = 12
n = np.arange(-P * N, P * N)
plt.plot(n, sqw(N, n));

### Exercise: a discrete-time triangular wave

A discrete-time triangular signal with period $N$ samples is defined over $[0, N-1]$ as
$$
    t[n] = \frac{4}{N}\left|n - \frac{N}{2}\right| - 1
$$
and prolonged by periodicity over $\mathbb{Z}$.

Complete the code below to implement a generator for the periodic triangular signal.

In [ ]:
def tri(N: int, n: np.ndarray=None) -> np.ndarray:
    n = np.arange(N) if n is None else n
    t = ...
    return t

In [ ]:
N = 64
plt.stem(tri(N));

In [ ]:
P = 10
n = np.arange(-P * N, P * N)
plt.plot(n, tri(N, n));

## Assembling signals from elementary building blocks

The general idea behind any of the transform used in signal processing is the ability to express a signal as a linear combinations of elements drawn from a set of atomic building blocks with specific properties. A very convenient way to look at this comes from the equivalence between this decomposition and performing a change of basis in a vector space; the atomic elements are the basis vectors and the coefficients in the new basis simply express the same information as in the original signal in a different reference system. Still, even without the machinery of linear algebra, we can develop a good intuition for the process using numerical examples.

### An elementary decomposition

Before introducing the sinusoidal elements used in Fourier analysis, consider the set $D_N = \{\boldsymbol{\delta}_k\}_{0\le k < N}$, where
$$
  \delta_k[n] = \begin{cases} 1 & n = k \\ 0 & n \neq k \end{cases}
$$
Each element in $D_N$ is a _time-selective_ element: the single nonzero value in $\boldsymbol{\delta}_k$ only encodes the information associated to the time index $n=k$.

In [ ]:
def delta(k:int, N:int) -> np.ndarray:
    d = np.zeros(N)
    d[k] = 1
    return d

In [ ]:
plt.figure(figsize=(12, 2))

N = 4
for k in range(N):
    plt.subplot(1, N, k+1)
    plt.stem(delta(k, N))
    plt.title(f'$\\boldsymbol{{\\delta}}_{k}$')
plt.tight_layout()

The set $D_N$ provides the simplest example of signal decomposition; indeed, it's obvious that any $N$-point signal $\mathbf{x}$ can be expressed as
$$
    \mathbf{x} = \sum_{k=0}^{N-1} d_k\, \boldsymbol{\delta}_k; 
$$
trivially, the coefficients of the linear combination are just the original signal values:
$$
    d_k = x[k].
$$

In [ ]:
# you can use any signal
x = sqw(10)

# reconstruct the signal using atomic time elements
N = len(x)
y = np.zeros(N)
for k in range(N):
    y += x[k] * delta(k, N)

plt.subplot(1,2,1)
plt.stem(x)
plt.title('$\\mathbf{{x}}$')
plt.subplot(1,2,2)
plt.stem(y)
plt.title(f'$\\sum t_k \\, \\boldsymbol{{\\delta}}_k$')
plt.tight_layout()

# Assembling signals from sinusoidal components

A signal decomposition using the _time-selective_ elements $\boldsymbol{\delta}_k$ is clearly self-evident but it's a useful starting point to understand the DFT. In Fourier analysis the goal remains the same: expressing a signal as a sum of atomic components; but now the elements are _frequency-selective_ instead, that is, each atomic component is an oscillation at a given frequency.

To develop an intuition for how this works, let's start by manually "assembling" the test signals we introduced earlier using a set of real-valued sinusoids. 

## Harmonic sines and cosines

Consider the family of discrete-time sine waves $S_N = \{\mathbf{s}_k\}_{0\le k < N}$ where
$$
    s_k[n] = \sin\left(\omega_k\,n\right) \qquad \omega_k = \frac{2\pi}{N}k
$$
The $k$-th element in the family is an oscillation with _angular frequency_ $\omega_k$ and, since all frequencies are integer multiples of the base frequency $2\pi/N$, the oscillations in $S_N$ are called _harmonic_.

Similarly, let $C_N = \{\mathbf{c}_k\}_{0\le k < N}$ be a family of $N$ harmonic _cosines_, where
$$
    c_k[n] = \cos\left(\omega_k\,n\right), \qquad \omega_k = \frac{2\pi}{N}k
$$

### Exercise: generators for harmonic sines and cosines 

Complete the function prototypes below so that they return the values of $s_k[n]$ and $c_k[n]$, respectively, for an arbitrary set of time values $n$

In [ ]:
def sin_N(N:int, k:int, n:np.ndarray=None) -> np.ndarray:
    # if n is not specified, return the values over the [0, N-1] interval
    n = np.arange(N) if n is None else n
    ... # your code here
    return s

def cos_N(N:int, k:int, n:np.ndarray=None) -> np.ndarray:
    # if n is not specified, return the values over the [0, N-1] interval
    n = np.arange(N) if n is None else n
    ... # your code here
    return c

Let’s check that it works by plotting a few sinusoids over the range $[0, N-1]$:

In [ ]:
N = 1000
for i, k in enumerate([1, 3, 5, 11]):
    # use different amplitude scaling to make the plot more readable
    plt.plot((2 - i/2) * sin_N(N, k))

### Building a square wave with sines

Although we won't formally derive it here, it's relatively easy to show that the square wave $\mathbf{q}$ can be built using the elements in $S_N$ as 
$$
    \mathbf{q} = \sum_{k=0}^{N-1} \alpha_k\, \mathbf{s}_k
$$
with
$$
    \alpha_k = \begin{cases} 0 & k \mathrm{~even} \\ \displaystyle \frac{2}{N\tan(\pi k /N)}  & k \mathrm{~odd} \end{cases}
$$
Perhaps surprisingly, it appears that a linear combination of smooth sine waves can reproduce a signal with abrupt amplitude jumps. 

Here is an implementation of the square wave generator using a sum of sines; we can verify numerically that the result is identical to the original square wave:

In [ ]:
def sqw_sines(N: int, n: np.ndarray=None, L: int=0) -> np.ndarray:
    n = np.arange(N) if n is None else n
    q = np.zeros_like(n).astype(float)   
    for k in range(1, N if L == 0 else L, 2):          
        a = 2 / (N * np.tan(np.pi / N * k))
        q += a * sin_N(N, k, n) 
    return q

In [ ]:
M = 10
np.allclose(sqw_sines(M), sqw(M))

In [ ]:
N, P = 64, 12
n = np.arange(-P * N, P * N)
plt.plot(n, sqw_sines(N, n));

By plotting partial sums we can appreciate how the square signal emerges incrementally as more sinusoidal components are added together; you should also note two things:
 * since all the even-indexed coefficients are zero, the partial sum only changes every 2 steps
 * it appears as if the shape of the square wave has already emerged after using only $N/2$ terms in the sum; this is not a coincidence and the reason will be clearer by the end of the notebook 

In [ ]:
def plot_partial(wave, N):
    def render(N, L):
        plt.figure(figsize=(8, 3));
        plt.plot(wave(N, L=L))
        plt.title(f"partial sum with {L} terms")
        plt.grid()
        plt.ylim(-1.1,1.1)

    return wg.interactive(render, wave=wg.fixed(wave), N=wg.fixed(N), L=wg.IntSlider(2, 2, N,2))

In [ ]:
plot_partial(sqw_sines, 200)

### Exercise: building a triangular wave with cosines

The triangular wave $\mathbf{t}$ can be expressed as a linear combination of harmonic cosines in $C_N$:
$$
    \mathbf{t} = \sum_{k=0}^{N-1} \beta_k\, \mathbf{c}_k
$$
where
$$
    \beta_k = \begin{cases} 0 & k \mathrm{~even} \\ \displaystyle \left[\frac{2}{N}\,\left(1 + \frac{1}{\tan(\pi k/N)}\right)\right]^2 & k \mathrm{~odd} \end{cases}
$$

Implement a generator for the triangle wave using harmonic cosines.

In [ ]:
def tri_cosines(N: int, n: np.ndarray=None, L: int=0) -> np.ndarray:
    n = np.arange(N) if n is None else n
    ...
    return t

In [ ]:
M = 1002
np.allclose(tri_cosines(M), tri(M))

In [ ]:
N, P = 64, 12
n = np.arange(-P * N, P * N)
plt.plot(n, tri_cosines(N, n));

In [ ]:
plot_partial(tri_cosines, 100)

## Symmetries

All the harmonic cosines in $C_N$ are symmetric signals whereas all the harmonic sines in $S_N$ are antisymmetric:
\begin{align}
    c_{k}[n] &= c_k[N-n] \\
    s_{k}[n] &= -s_k[N-n]
\end{align}
As a consequence, a signal generated using elements of $C_N$ will also be symmetric (as in the case of the triangular wave); similarly, a signal generated using only elements of $S_N$ will always be antisymmetric. If we want to build signals with no symmetries, we need to use elements from _both_ $C_N$ and $S_N$ in the linear combination:
$$
    \mathbf{x} = \sum_{k=0}^{N-1} \alpha_k\, \mathbf{s}_k + \sum_{k=0}^{N-1} \beta_k\, \mathbf{c}_k
$$

## How do we compute the coefficients?

So far, in the examples on how to build a signal with harmonic sines and cosines, the coefficients for the linear combination of sinusoidal elements have just been "given" without a derivation; and so, of course, the big question is: how can we compute the coefficients needed to generate an arbitrary signal? The answer is going to be provided by the DFT analysis formula.Before we get to it, however, let's introduce a new family of sinusoidal building blocks that will greatly simplify both the notation and the implementation of the DFT algorithms. 

As we just saw, to generate an arbitrary signal we need elements from $S_N$ and $C_N$ at the same time; but this makes the notation messy since we need to keep track of two distinct sums with different summation ranges and coefficients. Instead, it is much more convenient to use _complex-valued sinusoids_ as the elementary building blocks. There are three main advantages to this choice:
 * the notation becomes more compact than by using sines and cosines
 * trigonometric relations translate to simple complex algebra
 * a linear combination of complex-valued sinusoids can be used to build both real-valued or complex-valued signals.

Let $W_N = \{\mathbf{w}_k\}_{0 \le k < N}$ be the family of complex-valued harmonic oscillations of length $N$ defined as
$$
    w_k[n] = e^{j\omega_k n}, \quad \omega_k = \frac{2\pi}{N}k.
$$
Any length-$N$ signal $\mathbf{x}$ can be expressed as the linear combination
$$
    \mathbf{x} = \frac{1}{N} \sum_{k=0}^{N-1} X[k] \mathbf{w}_k;
$$
the vector of $N$ complex-valued coefficients $\mathbf{X}$ is called the Discrete Fourier Transform of $\mathbf{x}$ and it can be computed as 
$$
  \mathbf{X} = \sum^{N-1}_{n=0} x[n] \mathbf{w}^*_n.
$$

Let's unpack this one piece at a time

## The complex exponential

The expression $e^{j\theta}$ represents a point on the unit circle in the complex plane placed at an angle of $\theta$ radians measured _counterclockwise_ from the horizontal axis. The Cartesian coordinates of this point on the plane are its real and imaginary parts, that is, 
$$
    e^{j\theta} = cos\theta + j\sin\theta.
$$

Note that:
 * the position of the point is a $2\pi$-periodic function of the angle: $e^{j(\theta + 2m\pi)} = e^{j\theta}$ for any $m \in \mathbb{Z}$
 * if the angle is negative, the position of the point is measured _clockwise_ from the real axis
 * complex conjugation flips the sign of the angle: $(e^{j\theta})^* = e^{-j\theta}$

Because of the $2\pi$-periodicity, it is customary to _wrap_ the phase of a complex exponential over the interval $[-\pi, \pi]$; indeed, any point on the upper half of the unit circle can be reached via a **counterclockwise** rotation with $0 \le \theta \le \pi$ radians while points on the lower half can be reached via a **clockwise** rotation with $-\pi \le \theta \le 0$

In [ ]:
ws = wg.FloatSlider(value=0, min=-15, max=15, description='$\\theta$')
display_sbs(wg.interactive(ctspin, w=ws))

## Complex-valued harmonic oscillations

The family $W_N = \{\mathbf{w}_k\}_{0 \le k < N}$ is a set of $N$ discrete-time, complex-valued, harmonic oscillations where
$$
    w_k[n] = e^{j\omega_k n} = \cos(\omega_k n) + j \sin(\omega_k n), \qquad \omega_k = \frac{2\pi}{N}k
$$
The $N$ samples in $\mathbf{w}_k$ describe the position of a point on the complex plane that rotates _counterclockwise_ around the unit circle in angular increments of $\omega_k$ radians; the Cartesian coordinates of $w_k[n]$ are thus the cosine and the sine of the angle $\omega_k n$:

With NumPy, complex-valued computations are straightforward and we can generate the elements in $W_N$ as:

In [ ]:
def cex_N(N:int, k:int, n:np.ndarray=None) -> np.ndarray:
    n = np.arange(N) if n is None else n
    return np.exp(2j * np.pi / N * k * n)

The following interactive plot displays the samples in $\mathbf{w}_k$ as points on the complex plane and as plots of their real and imaginary parts (if multiple points fall on the same position, only the first label is shown):

In [ ]:
N = 16
display_sbs(wg.interactive(show_cex, N=wg.fixed(N), k=wg.IntSlider(1, 1, N-1)))

### Positive and negative frequencies

The complex values in $\mathbf{w}_k$ describe the trajectory of a point that rotates **counterclockwise** on the unit circle in discrete steps of $\omega_k = (2\pi/N)k$ radians. However, when $\omega_k > \pi$ (that is, when $k > N/2$), the trajectory is equivalent to a **clockwise** rotation with _negative_ step size $-(2\pi - \omega_k)$. Therefore:

 * for $0 \le k \le N / 2$ the frequency associated to $\mathbf{w}_k$ is positive, between zero and $\pi$; the complex oscillation rotates counterclockwise around the unit circle
 * for $N/2 < k < N$ the frequency associated to $\mathbf{w}_k$ is _negative_, between $\pi$ and zero; the complex oscillation rotates clockwise around the unit circle

The following interactive widget allows you get a feeling for this phenomenon. Start with $k$ small and move the slider for $n$ to see the counterclockwise movement around the unit circle. Now set $k$ around its maximum value and see how the sequence of points appear to be moving clockwise.

In [ ]:
N = 16
ks, ns = wg.IntSlider(1, 1, N-1), wg.IntSlider(1, 1, N-1)
def reset_n(*args):
    ns.value = 1
ks.observe(reset_n, 'value')
display_sbs(wg.interactive(dtspin, N=wg.fixed(N), n=ns, k=ks))

### Relationship to sines and cosines

Because of periodicity we have
$$
    w_{N-k}[n] = e^{j(2\pi/N)(N-k)n} = e^{-j\omega_k n} = (e^{j\omega_k n})^* = (w_{k}[n])^*
$$
or, in vector notation, remembering the index symmetries of harmonic sines and cosines,
$$
  \mathbf{w}_{N-k} = \mathbf{c}_{N-k} + j\,\mathbf{s}_{N-k} = \mathbf{c}_{k} - j\,\mathbf{s}_{k} = \mathbf{w}^*_k
$$

This means that every complex oscillation in $W_N$ has a "companion" spinning at the same angular speed but in the opposite direction; these complementary elements can be added and subtracted to obtain real-valued sinusoids:
\begin{align}
    \mathbf{w}_k + \mathbf{w}_{N-k} &= 2\,\mathbf{c}_k \\
    \mathbf{w}_k - \mathbf{w}_{N-k} &= 2j\,\mathbf{s}_k \\
\end{align}

The following widget allows you to visualize the relationship between the real and imaginary parts of $w_k[n]$ and $w_{N-k}[n]$; remember that if $k < N/2$ then $w_k[n]$ will be associated to a positive frequency $\omega_k$ while $w_{N-k}[n]$ to the _negative_ frequency $-\omega_k$

In [ ]:
display_sbs(wg.interactive(ccspin, N=wg.fixed(N), n=ns, k=ks))

## Building signals with the Fourier basis: the IDFT

Any length-$N$ discrete-time signal $\mathbf{x}$ can be expressed as a linear combination of $N$ harmonic complex exponentials:
$$
    \mathbf{x} = \frac{1}{N} \sum_{k=0}^{N-1} X[k] \mathbf{w}_k.
$$
This expression is called the _inverse DFT_; the (direct) DFT is the algorithm to compute the values of the Fourier coefficients $\mathbf{X}$. In this section we will just verify that we can use the IDFT to build our test signals.

### Symmetries

If $\mathbf{X}$ is a vector of complex-valued Fourier coefficients, let $X[k] = X_r[k] + j\,X_i[k]$. Here are two important cases:

 1. **real, symmetric coefficients:** If $X_r[k] = X_r[N-k]$ and $X_i[k] = 0$, then the imaginary parts of the complex oscillations will cancel out and the signal will be real and symmetric:
\begin{align}
    x[n] &= \frac{1}{N}\left( X_r[0] + \sum_{k=1}^{N/2-1} X_r[k] (w_k[n] + w_{N-k}[n]) + X_r[N-1]w_{N/2}[n]\right) \\
      &= \frac{1}{N}\left( X_r[0] + (-1)^nX_r[N-1] + \sum_{k=1}^{N/2-1} 2X_r[k] \, \cos(\omega_k n) \right) \in \mathbb{R}
\end{align}  
 1. **imaginary, antisymmetric coefficients:** If $X_i[k] = -X_i[N-k]$ and $X_r[k] = 0$, then the real parts of the complex oscillations will cancel out and the signal will be real and antisymmetric
\begin{align}
    x[n] &= \frac{1}{N}\sum_{k=1}^{N/2-1} j\,X_i[k] (w_k[n] - w_{N-k}[n]) \\
      &= -\frac{1}{N} \sum_{k=1}^{N/2-1} 2X_i[k] \, \sin(\omega_k n)  \in \mathbb{R}
\end{align}  

Therefore:
 * if the coefficients are real and symmetric the signal will be real and symmetric
 * if the coefficients are imaginary and antisymmetric, the signal will be real and antisymmetric

Similarly, if a signal is real-valued:
 * the real part of its Fourier coefficients will be symmetric
 * the imaginary part of its Fourier coefficients will be antisymmetric
 * the magnitude of its Fourier coefficients will be symmetric

### Example: gnerating sines and cosines

Consider an $N$-point vector $\mathbf{A}_i$ with only two complex-conjugate nonzero values:
$$
    A_i[k] = \begin{cases} (N/2)a & k = i \\ (N/2)a^* & k = N-i \\ 0 & \text{otherwise} \end{cases}
$$
According to the formulas we just derived, if we take the IDFT of $\mathbf{A}_i$ we should obtain
$$
    \mathrm{IDFT}\{\mathbf{A}_i\} = \begin{cases} \mathbf{c}_i & \text{if $a$ is real} \\ -\mathbf{s}_i & \text{if $a$ is imaginary} \end{cases}
$$

Let's verify this numerically; to compute the IDFT we will use NumPy's built-in `np.fft.ifft` routine.


In [ ]:
def testIDFT(N, i, a):
    A = np.zeros(N).astype(complex)
    A[i] = (N / 2) * a
    A[N-i] = (N / 2) * np.conj(a)
    x = np.fft.ifft(A)
    plt.plot(cos_N(N, i), label=f'$\\mathbf{{c}}_{{{i}}}$')
    plt.plot(-sin_N(N, i), label=f'$-\\mathbf{{s}}_{{{i}}}$')
    plt.plot(x.real, 'o', label=f'IDFT{{$\\mathbf{{A}}_{{{i}}}$}}')  
    plt.legend()

In [ ]:
testIDFT(64, 3, 1)

In [ ]:
testIDFT(64, 5, 1j)

### Example: square wave revisited

The square wave is antisymmetric so its expression using the IDFT will be
$$
    \mathbf{q} = \frac{1}{N} \sum_{k=0}^{N-1} Q[k] \mathbf{w}_k = \sum_{k=1}^{N/2-1} -\frac{2}{N}Q_i[k] \, \mathbf{s}_k
$$
By comparing this to the earlier expression using sines we can derive the Fourier coefficients for the square wave: 
$$
    Q[k] = \begin{cases} 0 & k \text{~even} \\ \displaystyle \frac{-2j}{\tan(\pi k /N)}  & k \text{~odd} \end{cases}
$$

In [ ]:
def sqw_idft(N: int, n: np.ndarray=None) -> np.ndarray:
    n = np.arange(N) if n is None else np.mod(n, N)
    q = np.fft.ifft([-2j / np.tan(np.pi / N * k) if k % 2 == 1 else 0 for k in range(N)]).real
    return q[n]

In [ ]:
M = 1002
np.allclose(sqw_idft(M), sqw(M))

### Exercise: triangular wave revisited

Complete the code below to generate a triangular wave using the IDFT; the Fourier coefficients are 
$$
    T[k] = \begin{cases} 0 & k \text{~even} \\ \displaystyle \frac{4}{N}\,\left(1 + \frac{1}{\tan^2(\pi k/N)}\right) & k \text{~odd} \end{cases}
$$

In [ ]:
def tri_idft(N: int, n: np.ndarray=None) -> np.ndarray:
    n = np.arange(N) if n is None else np.mod(n, N)
    ... # your code here
    return t[n]

Using the same reasoning as before, but considering that the triangular wave is symmetric, we can write


In [ ]:
M = 14
np.allclose(tri_idft(M), tri(M))

## Parity

So far we have always assumed that all signals have _even_ length $N = 2M$; in this case, the signal's midpoint $N/2 = M$ is also an integer. If $N$ is odd, so that $N = 2M+1$, everything we discussed remains valid as long as we replace the non-integer value $N/2$ with the integer index $(N-1)/2 = M$. 

In terms of the Fourier basis, the main difference between $N$ even and odd is that, for $N$ even, the set $W_{N}$ contains an oscillation at the maximum discrete-time frequency $\pi$ which is not present for $N$ odd. Indeed, if $N=2M$,
$$
    w_{M}[n] = e^{j\frac{2\pi}{2M}Mn} = e^{j\pi n} = (-1)^n
$$
Note that this is consistent with the index symmetry of the Fourier elements given by $\mathbf{w}_{N-k} = \mathbf{w}^*_k$; for $k=M$ this becomes $\mathbf{w}_{M} = \mathbf{w}^*_M$ which requires $\mathbf{w}_{M}$ to be real-valued. 

The following plot shows the $N$ angular frequencies $\omega_k$ associated to the elements of $W_{N}$ and $W_{N+1}$:

In [ ]:
def odd_even(N):
    plt.subplot(1,2,1)
    show_freqs(N)
    plt.subplot(1,2,2)
    show_freqs(N+1)

display_sbs(wg.interactive(odd_even, N=wg.IntSlider(3, 3, 20)))

Just as for the frequency $\omega_0 = 1$, the frequency $\omega = \pi$ can be interpreted indifferently as positive or negative since
$$
    e^{j\pi n} = e^{-j\pi n}.
$$
This will be important in the second part of this notebook, when we address the question of how to plot the set Fourier coefficients